In [1]:
import snowflake.connector
import os
import pandas as pd
import re

WH =''
DB =''

ctx = snowflake.connector.connect(user='',
password='',
account='',
host='',
authenticator='externalbrowser',
warehouse='')


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [2]:
### Import ###

from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw()

data_path = askopenfilename()
print(data_path)


C:/Users/jamielai/Desktop/FG_box.txt


In [3]:
### read lot list ###
 
f = open(data_path, 'r')
LT1 = f.read().split('\n')
LT1 = list(dict.fromkeys(LT1)) #dict keys can remove duplicate
while("" in LT1):
    LT1.remove("") #drop na    
LT1=tuple(LT1) #change to tuple
 
cs = ctx.cursor() 
 
query = '''
SELECT DISTINCT FROM_LOT_ID, TO_LOT_ID, STEP 
FROM PROD_BEMFG_DM.LOT. LOT_HIERARCHY_INFO
WHERE FROM_LOT_ID  IN {} 
'''.format(LT1)
 
cs.execute(query)
df_L1 = cs.fetch_pandas_all()
df_L1['STEP'] = df_L1['STEP'].fillna('NULL') #Step = NULL

LT2 = tuple(df_L1 ['TO_LOT_ID'].dropna().drop_duplicates())

n = 20
for i in range(n):
    
    print(i)
    
    query = '''
    SELECT DISTINCT FROM_LOT_ID, TO_LOT_ID, STEP 
    FROM PROD_BEMFG_DM.LOT. LOT_HIERARCHY_INFO
    WHERE FROM_LOT_ID  IN {} 
    '''.format(LT2)

    cs.execute(query)
    df_L2 = cs.fetch_pandas_all()
    df_L2['STEP'] = df_L2['STEP'].fillna('NULL') #Step = NULL

    df_L1 = df_L1.merge(df_L2, left_on=["TO_LOT_ID"], right_on=["FROM_LOT_ID"], how='left')
    df_L1 = df_L1.rename(columns={'FROM_LOT_ID_y': 'FROM_LOT_ID', 'TO_LOT_ID_y': 'TO_LOT_ID', 'STEP_y': 'STEP'})

    
    indx = df_L1[ df_L1.loc[:,'FROM_LOT_ID'] == df_L1.loc[:,'TO_LOT_ID'] ].index #FROM==TO index
    df_temp = df_L1.drop(indx) #drop those index

    # A is current lot list to query, B is all the FROM_LOT_ID in past iterations
    A = set(df_temp['TO_LOT_ID'].dropna().drop_duplicates())
    
    if df_L1.shape[1]==6:
        B = set(df_L1['FROM_LOT_ID_x']) #for n=0
    else:
        B = set(df_L1['FROM_LOT_ID_x'].stack()) #for n>=1
        
    LT2 = A-B #remove the lots already querry in past iterations
    LT2 = tuple(LT2)

    if len(LT2) == 1:
        LT2 = (LT2[0],LT2[0]) #to avoid tuple with only one element will error in SQL query

    if len(LT2) == 0:
        break #loop end when no more new TO lot IDs

        

def org(x):
    return x.dropna()[0]

def last(x):
    return x.dropna()[-3]

def step(x):
    return x.dropna()[-3]


df_L1["ORIGIN_FROM_LOT_ID"] = df_L1.apply(org, axis=1)
df_L1["LAST_TO_LOT_ID"] = df_L1.apply(last, axis=1)
df_L1["LAST_STEP"] = df_L1.apply(step, axis=1)

#df_L1.rename(columns={'STEP': 'LAST_STEP'}, inplace=True)


0


ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 3 at position 27 unexpected ')'.

In [7]:
### save file
from tkinter import Tk
from tkinter.filedialog import asksaveasfilename
filename = asksaveasfilename(filetypes = [('All Files', '*')])
df_L1.to_csv(filename, index=False)